In [1]:
import boto3
import json
from datetime import datetime
from edc import setup_environment_variables

# Oauth
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

setup_environment_variables()

API credentials have automatically been injected for your active subscriptions.  
The following environment variables are now available:
* `SH_CLIENT_NAME`
* `SH_CLIENT_SECRET`
* `SH_INSTANCE_ID`
* `SH_CLIENT_ID`

The following additional environment variables have been loaded from `~/custom.env`:
* `AWS_BUCKET`
------


In [2]:
# Your client credentials
client_id = %env SH_CLIENT_ID
client_secret = %env SH_CLIENT_SECRET

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

url = 'https://services.sentinel-hub.com/batch/v1/process/'

In [3]:
bucket_name = %env AWS_BUCKET

In [4]:
i = 0

s3 = boto3.resource('s3')
bk = s3.Bucket(bucket_name)

objects = list(bk.objects.all())
request_ids = set([o.key.split('/')[0] for o in objects if '/' in o.key])

for rid in request_ids:
    # Check if measurements.json already exists for a request
    if f'{rid}/measurements.json' in [o.key for o in objects]:
        pass
    else:
        response = oauth.request('GET', f'{url}{rid}')

        if response.ok:
            request = response.json()

            if request['status'] == 'DONE':
                # Collect size
                size = sum([o.size for o in objects if o.key.startswith(rid)])
                
                # Collect execution time
                created = datetime.fromisoformat(request['created'].replace('Z','+00:00'))
                finished = max([o.last_modified for o in objects if o.key.startswith(rid) and
                               o.key.endswith('tif')])
                
                # Collect costs
                tiles = []
                viewtoken = 0
                while viewtoken is not None:
                    response = oauth.request('GET', f'{url}{rid}/tiles?viewtoken={viewtoken}').json()
                    tiles.extend(response['member'])
                    viewtoken=response['view']['nextToken']
                costs = sum([t['cost'] for t in tiles])

                # Save measurements to bucket
                bk.put_object(Key=rid + '/measurements.json', Body=json.dumps({
                    'description': request['description'],
                    'id': rid,
                    'execution time': str(finished-created),
                    'costs': costs,
                    'n_tiles': len(tiles),
                    'n_timeslices': len(json.load(bk.Object(key=f'{rid}/userdata.json').get()['Body'])['time']),
                    'size': size
                }))
                
                i+=1


print(i, 'measurements written to bucket')

0 measurements written to bucket


In [5]:
import pandas as pd
objects = list(bk.objects.all())
df= pd.DataFrame([json.load(o.get()['Body']) for o in objects if o.key.endswith('measurements.json')])
df['size'] = df['size'] /2**30 #GiB
df[(df.description.str.contains('Test MAB'))]#.plot()

,description,id,execution time,costs,size,n_tiles,n_timeslices
51,Test MAB 20170901_20170930,59240413-4e97-4023-ac04-81da91eb5e71,0:08:30.490000,20818.306003,7.327729,NaN,NaN
125,Test MAB 20170901_20170930,d0382f75-3b67-4ed7-8481-b6393d6cdb64,0:08:51.300000,21729.555494,7.328047,57.0,2.0
